In [1]:
import pandas as pd
import talib
import random

In [2]:
data_df = pd.read_csv("data/LTCUSDT-30m-data.csv")

In [3]:
data_df

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-12-13 03:30:00.000,272.00,288.00,272.00,276.00,130.86075,1513137599999,3.623134e+04,65,11.87562,3.319173e+03,265049.598083
1,2017-12-13 04:00:00.000,276.00,284.10,269.99,270.00,350.86428,1513139399999,9.616079e+04,77,44.46382,1.216524e+04,264975.004646
2,2017-12-13 04:30:00.000,270.00,288.00,270.00,279.00,255.61702,1513141199999,7.056886e+04,88,92.45727,2.570841e+04,264696.549732
3,2017-12-13 05:00:00.000,279.00,320.00,279.00,287.00,220.08266,1513142999999,6.384420e+04,82,148.75093,4.300407e+04,262766.913647
4,2017-12-13 05:30:00.000,287.00,330.00,282.00,302.00,163.68408,1513144799999,4.916537e+04,87,89.86945,2.767100e+04,261070.341130
...,...,...,...,...,...,...,...,...,...,...,...,...
50958,2020-11-13 07:00:00,62.55,62.55,61.58,61.61,26435.73662,1605252599999,1.643066e+06,3487,9861.91699,6.133267e+05,0.000000
50959,2020-11-13 07:30:00,61.64,62.40,61.25,62.30,45162.19962,1605254399999,2.788383e+06,5393,22479.94228,1.388100e+06,0.000000
50960,2020-11-13 08:00:00,62.27,62.56,61.54,61.92,33160.95155,1605256199999,2.055975e+06,4537,16447.27198,1.019831e+06,0.000000
50961,2020-11-13 08:30:00,61.92,63.67,61.73,63.54,55368.46795,1605257999999,3.480437e+06,7976,34112.11884,2.146223e+06,0.000000


In [4]:
params = {
    'RSI_BULL': 10,
    'RSI_BULL_TF': 19,
    'RSI_BULL_HIGH': 81.93600697495238,
    'RSI_BULL_LOW' : 45.516632226052714,
    'RSI_BEAR': 15,
    'RSI_BEAR_TF': 16,
    'RSI_BEAR_HIGH': 45.61985405602511,
    'RSI_BEAR_LOW' : 20.925647327774335,
    'SMA_FAST': 138,
    'SML_SLOW': 67,
    'SMA_TF': 6,
    'ADX': 3,
    'ADX_TF': 9,
    'ADX_HIGH': 65.80525318969225,
    'ADX_LOW': 45.6513373850286,
    'BULL_MOD_HIGH': 5,
    'BULL_MOD_LOW' : -5,
    'BEAR_MOD_HIGH': 15,
    'BEAR_MOD_LOW' : -5,
    'STOP_LOSS': 20, # %,
    'TAKE_PROFIT': 14 # %
}

params2 = {'RSI_BULL': 14, 'RSI_BULL_TF': 6, 'RSI_BULL_HIGH': 70.66602096693327, 'RSI_BULL_LOW': 47.72774022537839, 'RSI_BEAR': 19, 'RSI_BEAR_TF': 20, 'RSI_BEAR_HIGH': 47.831780740039356, 'RSI_BEAR_LOW': 19.675569028578884, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 9, 'ADX': 15, 'ADX_TF': 5, 'ADX_HIGH': 62.76900764282739, 'ADX_LOW': 30.616489167985236, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 12.475810949883751, 'TAKE_PROFIT': 10.09271064841754}
params3 = {'RSI_BULL': 16, 'RSI_BULL_TF': 20, 'RSI_BULL_HIGH': 71.15720121829733, 'RSI_BULL_LOW': 47.5028232805205, 'RSI_BEAR': 18, 'RSI_BEAR_TF': 11, 'RSI_BEAR_HIGH': 38.73422848226052, 'RSI_BEAR_LOW': 20.191061873397032, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 8, 'ADX': 6, 'ADX_TF': 16, 'ADX_HIGH': 90.73868788251929, 'ADX_LOW': 25.35677072649313, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 13.250081908012456, 'TAKE_PROFIT': 16.4993153742895}
params4 = {'RSI_BULL': 18, 'RSI_BULL_TF': 8, 'RSI_BULL_HIGH': 65.61862224052749, 'RSI_BULL_LOW': 43.02113912133937, 'RSI_BEAR': 15, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 58.39183822274852, 'RSI_BEAR_LOW': 12.656621672290465, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 5, 'ADX': 5, 'ADX_TF': 16, 'ADX_HIGH': 77.57150575486634, 'ADX_LOW': 56.41056770151446, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.8330569920731217, 'TAKE_PROFIT': 31.093215318561775}
params5 = {'RSI_BULL': 18, 'RSI_BULL_TF': 14, 'RSI_BULL_HIGH': 70.98853186912616, 'RSI_BULL_LOW': 46.045622334549414, 'RSI_BEAR': 16, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 57.46995465236236, 'RSI_BEAR_LOW': 24.005702805940228, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 10, 'ADX_TF': 13, 'ADX_HIGH': 81.64531842257551, 'ADX_LOW': 21.6834731576957, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.3089068361825446, 'TAKE_PROFIT': 36.159684423955504}

In [5]:
def run_strategy(data, params):
    # calculating
    rsi_bull = talib.RSI(data['close'], timeperiod=params['RSI_BULL'])
    rsi_bear = talib.RSI(data['close'], timeperiod=params['RSI_BEAR'])
    sma_slow = talib.SMA(data['close'], timeperiod=params['SML_SLOW'])
    sma_fast = talib.SMA(data['close'], timeperiod=params['SMA_FAST'])
    adx = talib.ADX(high=data['high'], low=data['low'], close=data['close'], timeperiod=params['ADX'])

    order_history = []
    entry_price = 0
    exit_price = 0
    dd = 0
    ru = 0
    dd_percent = 0
    ru_percent = 0

    sma_timeframe = params['SMA_TF']
    adx_timeframe = params['ADX_TF']
    rsi_bull_timeframe = params['RSI_BULL_TF']
    rsi_bear_timeframe = params['RSI_BEAR_TF']
    rsi_bull_high = params['RSI_BULL_HIGH']
    rsi_bull_low = params['RSI_BULL_LOW']
    rsi_bear_high = params['RSI_BEAR_HIGH']
    rsi_bear_low = params['RSI_BEAR_LOW']
    adx_high = params['ADX_HIGH']
    adx_low = params['ADX_LOW']
    bull_mod_high = params['BULL_MOD_HIGH']
    bull_mod_low = params['BULL_MOD_LOW']
    bear_mod_high = params['BEAR_MOD_HIGH']
    bear_mod_low = params['BEAR_MOD_LOW']
    stop_loss = params['STOP_LOSS']  # %
    take_profit = params['TAKE_PROFIT']  # %

    sma_slow_count = 0
    sma_fast_count = 0
    adx_count = 0
    rsi_bull_count = 0
    rsi_bear_count = 0

    in_position = False
    in_fake_position = False  # position if dont have stop loss/take profit
    sma_fast_value = 0
    sma_slow_value = 0
    adx_value = 0

    for index, row in data.iterrows():
        price = row['close']
        # SMA slow
        if sma_slow_count >= sma_timeframe:
            sma_slow_value = sma_slow[index]
            sma_slow_count = 0
        else:
            sma_slow_count += 1

        # SMA fast
        if sma_fast_count >= sma_timeframe:
            sma_fast_value = sma_fast[index]
            sma_fast_count = 0
        else:
            sma_fast_count += 1

        # ADX
        if adx_count >= adx_timeframe:
            adx_value = adx[index]
            adx_count = 0
        else:
            adx_count += 1

        # RSI bull
        rsi_bull_value = rsi_bull[index]

        # RSI bear
        rsi_bear_value = rsi_bear[index]

        is_bull_market = sma_fast_value > sma_slow_value
        is_bear_market = sma_fast_value < sma_slow_value

        # dd, ru
        if in_position:
            dd = min(dd, price)
            dd_percent = 100 * (dd / price - 1)
            ru = max(ru, price)
            ru_percent = 100 * (ru / price - 1)
            # stop/take profit
            if (1 - price / entry_price) > stop_loss / 100 or (price / entry_price - 1) > take_profit / 100:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False
        else:
            dd = 0
            ru = 0
            dd_percent = 0
            ru_percent = 0

        if is_bull_market:
            if adx_value > adx_high:
                rsi_bull_high_adx = rsi_bull_high + bull_mod_high
            elif adx_value < adx_low:
                rsi_bull_low_adx = rsi_bull_low + bull_mod_low
            if not in_position and rsi_bull_value >= rsi_bull_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bull_value <= rsi_bull_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False

        elif is_bear_market:
            if adx_value > adx_high:
                rsi_bear_high_adx = rsi_bear_high + bear_mod_high
            elif adx_value < adx_low:
                rsi_bear_low_adx = rsi_bear_low + bear_mod_low
            if not in_position and rsi_bear_value >= rsi_bear_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bear_value <= rsi_bear_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1),
                     "draw_down": dd, "run_up": ru, "draw_down_percent": dd_percent, "run_up_percent": ru_percent})
                in_position = False
    return pd.DataFrame(order_history)

In [7]:
order_history_df = run_strategy(data_df, params)

In [8]:
order_history_df

,entry,exit,profit,draw_down,run_up,draw_down_percent,run_up_percent
0,303.20,355.00,17.084433,282.05,355.00,-20.549296,0.000000
1,355.00,309.98,-12.681690,289.00,363.61,-6.768179,17.301116
2,250.00,262.69,5.076000,238.02,274.12,-9.391298,4.351136
3,271.52,242.00,-10.872127,236.51,283.50,-2.268595,17.148760
4,220.64,255.96,16.007977,219.47,255.96,-14.256134,0.000000
...,...,...,...,...,...,...,...
244,54.40,56.65,4.136029,52.98,59.65,-6.478376,5.295675
245,58.31,55.60,-4.647573,54.88,58.31,-1.294964,4.874101
246,55.33,53.90,-2.584493,53.51,56.54,-0.723562,4.897959
247,55.53,64.11,15.451108,55.11,64.11,-14.038372,0.000000


In [9]:
order_history_df.to_csv("LTCUSD-30m-results.csv")

In [10]:
def performance_report(order_df):
    total_profit = order_df['profit'].sum()
    max_dd = order_df['draw_down_percent'].min()
    max_ru = order_df['run_up_percent'].max()
    num_win = order_df[order_df.profit >= 0].count().profit
    num_loss = order_df[order_df.profit < 0].count().profit
    total_win_profit = order_df[order_df.profit >= 0].sum().profit
    total_loss_profit = order_df[order_df.profit < 0].sum().profit
    ave_win = total_win_profit / num_win
    ave_loss = total_loss_profit / num_loss
    win_rate = num_win / (num_win + num_loss)
    start_price = order_df.iloc[0]['entry']
    end_price = order_df.iloc[-1]['exit']
    market = (end_price / start_price - 1) * 100
    start_balance = 100
    balance = start_balance * (1 + total_profit / 100)
    relative_profit = balance - start_balance
    relative_yearly_profit = relative_profit / 3
    sharpe = 10
    trades = order_df.shape[0] * 2

    return {'profit': float(total_profit), 'max_dd': float(max_dd), 'max_ru': float(max_ru),
            'num_win': int(num_win), 'num_loss': int(num_loss),
            'total_win_profit': int(total_win_profit),
            'total_loss_profit': int(total_loss_profit),
            'ave_win': float(ave_win), 'ave_loss': float(ave_loss),
            'win_rate': float(win_rate), 'balance': float(balance), 'startBalance': float(start_balance),
            'relativeYearlyProfit': float(relative_yearly_profit), 'market': float(market), 'sharpe': float(sharpe),
            'trades': int(trades), 'startPrice': start_price, 'endPrice': end_price, 'relativeProfit': relative_profit}


In [11]:
performance_report(order_history_df)

{'profit': 136.049116180779,
 'max_dd': -20.54929577464788,
 'max_ru': 29.126722588376253,
 'num_win': 92,
 'num_loss': 157,
 'total_win_profit': 798,
 'total_loss_profit': -662,
 'ave_win': 8.677180709392522,
 'ave_loss': -4.218162478237792,
 'win_rate': 0.36947791164658633,
 'balance': 236.04911618077898,
 'startBalance': 100.0,
 'relativeYearlyProfit': 45.34970539359299,
 'market': -80.15501319261213,
 'sharpe': 10.0,
 'trades': 498,
 'startPrice': 303.2,
 'endPrice': 60.17,
 'relativeProfit': 136.04911618077898}

In [84]:
# backtest
# Brute force
total_profit = 0
while True:
    params = {
        'RSI_BULL': random.randint(5, 20),
        'RSI_BULL_TF': random.randint(5, 20),
        'RSI_BULL_HIGH': random.uniform(60, 100),
        'RSI_BULL_LOW' : random.uniform(40, 60),
        'RSI_BEAR': random.randint(5, 20),
        'RSI_BEAR_TF': random.randint(5, 20),
        'RSI_BEAR_HIGH': random.uniform(30, 60),
        'RSI_BEAR_LOW' : random.uniform(5, 30),
        'SMA_FAST': 138,
        'SML_SLOW': 67,
        'SMA_TF': random.randint(5, 20),
        'ADX': random.randint(5, 20),
        'ADX_TF': random.randint(5, 20),
        'ADX_HIGH': random.uniform(60, 100),
        'ADX_LOW': random.uniform(5, 60),
        'BULL_MOD_HIGH': 5,
        'BULL_MOD_LOW' : -5,
        'BEAR_MOD_HIGH': 15,
        'BEAR_MOD_LOW' : -5,
        'STOP_LOSS': random.uniform(0.1, 10), # %,
        'TAKE_PROFIT': random.uniform(0.2, 50) # %
    }
    odf = run_strategy(data_df, params)
    pr  = performance_report(odf)
    if total_profit <= pr['total_profit']:
        total_profit = pr['total_profit']
        print("------------------------")
        print ("Best profit: ", total_profit)
        print ("Params: ", params)
        print("------------------------")

------------------------
Best profit:  8.490561479272394
Params:  {'RSI_BULL': 7, 'RSI_BULL_TF': 9, 'RSI_BULL_HIGH': 86.7702430737971, 'RSI_BULL_LOW': 49.81543734719503, 'RSI_BEAR': 15, 'RSI_BEAR_TF': 16, 'RSI_BEAR_HIGH': 41.186620272061305, 'RSI_BEAR_LOW': 12.940871526214906, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 6, 'ADX': 17, 'ADX_TF': 14, 'ADX_HIGH': 82.44364789394751, 'ADX_LOW': 53.06345022385097, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 6.23110973164169, 'TAKE_PROFIT': 36.48315991576723}
------------------------
------------------------
Best profit:  17.118282824408897
Params:  {'RSI_BULL': 5, 'RSI_BULL_TF': 7, 'RSI_BULL_HIGH': 92.81793542838311, 'RSI_BULL_LOW': 59.57681507376149, 'RSI_BEAR': 13, 'RSI_BEAR_TF': 17, 'RSI_BEAR_HIGH': 57.00039370970774, 'RSI_BEAR_LOW': 12.526749402745711, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 18, 'ADX': 20, 'ADX_TF': 9, 'ADX_HIGH': 82.52905165303783, 'ADX_LOW': 37.82231501227939, 'BULL_MO

KeyboardInterrupt: 

In [41]:
# backtest
# GA
from pyeasyga import pyeasyga

# setup data
data = data_df

ga = pyeasyga.GeneticAlgorithm(data)        # initialise the GA with data

def create_individual(data):
    individual = [
        random.randint(5, 20),
        random.randint(5, 20),
        random.uniform(60, 100),
        random.uniform(40, 60),
        random.randint(5, 20),
        random.randint(5, 20),
        random.uniform(30, 60),
        random.uniform(5, 30),
        random.randint(70, 200),
        random.randint(5, 70),
        random.randint(5, 20),
        random.randint(2, 10),
        random.randint(5, 10),
        random.uniform(60, 100),
        random.uniform(5, 60),
        5,
        -5,
        15,
        -5,
        random.uniform(0.1, 10), # %,
        random.uniform(0.2, 50) # %
    ]
    return individual

ga.create_individual = create_individual

# define a fitness function
def fitness(individual, data):
#     print (individual)
    params = {
        'RSI_BULL': individual[0],
        'RSI_BULL_TF': individual[1],
        'RSI_BULL_HIGH': individual[2],
        'RSI_BULL_LOW' : individual[3],
        'RSI_BEAR': individual[4],
        'RSI_BEAR_TF': individual[5],
        'RSI_BEAR_HIGH': individual[6],
        'RSI_BEAR_LOW' : individual[7],
        'SMA_FAST': individual[8],
        'SML_SLOW': individual[9],
        'SMA_TF': individual[10],
        'ADX': individual[11],
        'ADX_TF': individual[12],
        'ADX_HIGH': individual[13],
        'ADX_LOW': individual[14],
        'BULL_MOD_HIGH': individual[15],
        'BULL_MOD_LOW' : individual[16],
        'BEAR_MOD_HIGH': individual[17],
        'BEAR_MOD_LOW' : individual[18],
        'STOP_LOSS': individual[19], # %,
        'TAKE_PROFIT': individual[20] # %
    }
#     params = {'RSI_BULL': 18, 'RSI_BULL_TF': 14, 'RSI_BULL_HIGH': 70.98853186912616, 'RSI_BULL_LOW': 46.045622334549414, 'RSI_BEAR': 16, 'RSI_BEAR_TF': 8, 'RSI_BEAR_HIGH': 57.46995465236236, 'RSI_BEAR_LOW': 24.005702805940228, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 15, 'ADX': 10, 'ADX_TF': 13, 'ADX_HIGH': 81.64531842257551, 'ADX_LOW': 21.6834731576957, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 0.3089068361825446, 'TAKE_PROFIT': 36.159684423955504}
#     params['STOP_LOSS'] = individual[19]
    odf = run_strategy(data, params)
    pr  = performance_report(odf)
    print ("----------------")
    print ("Profit: ",pr['total_profit'])
    return float(pr['total_profit'])

ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print (ga.best_individual())                 # print the GA's best solution

----------------
Profit:  -81.64430633671773
----------------
Profit:  -35.22827160322875
----------------
Profit:  29.129080627035655
----------------
Profit:  -78.21283539147771
----------------
Profit:  -53.31857336506793
----------------
Profit:  -28.520179042958414
----------------
Profit:  -115.7829102745516
----------------
Profit:  29.36168536657781
----------------
Profit:  7.370989789696456
----------------
Profit:  -76.13239967018566
----------------
Profit:  -37.760054487088816
----------------
Profit:  38.16031732021988
----------------
Profit:  -129.96276832737297
----------------
Profit:  -87.82845851726844
----------------
Profit:  -140.13498957633215
----------------
Profit:  -115.03080492523213
----------------
Profit:  -128.28484987248154
----------------
Profit:  -38.12983232030214
----------------
Profit:  -51.65655854062649
----------------
Profit:  37.39866335299159
----------------
Profit:  -72.25971212688529
----------------
Profit:  -36.86953010472265
--------

KeyboardInterrupt: 

In [33]:
pr

{'total_profit': -119.67190574024251,
 'max_dd': 23.12,
 'max_ru': 363.61,
 'num_win': 90,
 'num_loss': 202,
 'total_win_profit': 552.4039082928518,
 'total_loss_profit': -672.075814033094,
 'ave_win': 6.1378212032539095,
 'ave_loss': -3.3271079902628418,
 'win_rate': 0.3082191780821918}